In [1]:
import pandas as pd
import numpy as np

from math import floor, ceil

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
N_TOTAL_SEATS = 500

In [3]:
df = pd.read_csv('../inputs/2019-general-election.csv', index_col="party")
df

,n_votes,n_regional_mp
party,,
เพื่อไทย,7881006,136
พลังประชารัฐ,8441274,97
อนาคตใหม่,6330617,31
ประชาธิปัตย์,3959358,33
ภูมิใจไทย,3734459,39
เสรีรวมไทย,824284,0
ชาติไทยพัฒนา,783689,6
ประชาชาติ,481490,6
เศรษฐกิจใหม่,486273,0


In [4]:
N_CONSTITUENCY_SEATS = df["n_constituency_seats"].sum()
N_PROPORTION_SEATS = N_TOTAL_SEATS - N_CONSTITUENCY_SEATS

N_CONSTITUENCY_SEATS, N_PROPORTION_SEATS

KeyError: 'n_constituency_seats'

In [ ]:
# มาตรา ๑๒๘ เมื่อคณะกรรมการได้รับรายงานผลรวมคะแนนแบบบัญชีรายชื่อจาก
# ผู้อำนวยการการเลือกตั้งประจำจังหวัดแล้ว ให้ดำเนินการคำนวณหาจำนวนสมาชิกสภาผู้แทนราษฎร
# ที่พรรคการเมืองจะพึงมีได้
# และให้ดำเนินการคำนวณเพื่อหาผู้ที่ได้รับเลือกตั้งเป็นสมาชิกสภาผู้แทนราษฎร
# แบบบัญชีรายชื่อ ดังต่อไปนี้โดยในกรณีที่มีเศษให้ใช้ทศนิยมสี่ตำแหน่ง

def use_four_decimal_place(n): return round(n, 4)
# def use_four_decimal_place(n): return n - (n % 0.0001)

In [ ]:
# (๑) ให้รวมผลคะแนนทั้งหมดที่ทุกพรรคการเมืองได้รับจากการเลือกตั้งแบบบัญชีรายชื่อ
# ทั้งประเทศ

total_votes = df["n_votes"].sum()
total_votes

In [ ]:
# (๒) ให้นำคะแนนรวมจาก (๑) หารด้วยหนึ่งห้าร้อยอันเป็นจำนวนสมาชิกทั้งหมด
# ของสภาผู้แทนราษฎร ผลลัพธ์ที่ได้ให้ถือเป็นคะแนนเฉลี่ยต่อสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ
# พึงมีหนึ่งคน
score_required_per_rep = total_votes / 500
score_required_per_rep

In [ ]:
# Filter non-party records

df = df[df["is_party"] == 1]
df

In [ ]:
# (๓) นำผลลัพธ์ตาม (๒) ไปหารจำนวน คะแนนรวมทั้งประเทศที่ได้รับจากการเลือกตั้ง
# แบบบัญชีรายชื่อของแต่ละพรรค ผลลัพธ์ที่ได้ให้ถือเป็นจำนวนสมาชิกสภาผู้แทนราษฎร
# ที่พรรคการเมืองนั้นจะพึงมีได้เบื้องต้น และเมื่อได้คำนวณตาม (๖) (๗) หรือ (๘) ถ้ามีแล้ว 
# จึงให้ถือว่าเป็นจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้

# `step_3_base` will be "จำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้เบื้องต้น"

df['step_3'] = use_four_decimal_place(df["n_votes"] / score_required_per_rep)
df

In [ ]:
# (๔) นำจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมือง
# จะพึงมีได้ตาม (๓) ลบด้วยจำนวนสมาชิกสภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้ง
# ทั้งหมดที่พรรคการเมืองนั้นได้รับเลือกตั้งในทุกเขตเลือกตั้ง
# ผลลัพธ์ที่ได้คือจำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อพรรคการเมืองนั้นจะได้รับเบื้องต้น

df["step_4"] = df["step_3"] - df["n_constituency_seats"]
df

In [ ]:
# (๖) ถ้าพรรคการเมืองใดมีผู้ได้รับเลือกตั้งเป็นสมาชิกสภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้ง
# เท่ากับหรือสูงกว่าจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้ตาม (๓)
# ให้พรรคการเมืองนั้นมีสมาชิกสภาผู้แทนราษฎรตามจำนวนที่ได้รับการเลือกตั้งแบบแบ่งเขตเลือกตั้ง 
# และไม่มีสิทธิได้รับการจัดสรรสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ และให้นำจำนวนสมาชิก
# สภาผู้แทนราษฎรแบบบัญชีรายชื่อทั้งหมดไปจัดสรรให้แก่พรรคการเมืองที่มีจำนวนสมาชิก
# สภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้งต่ำกว่าจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้
# ตาม (๓) ตามอัตราส่วน แต่ต้องไม่มีผลให้พรรคการเมืองใดดังกล่าวมีสมาชิกสภาผู้แทนราษฎร
# เกินจำนวนที่จะพึงมีได้ตาม (๓)

df["step_6_eligible_for_proportion"] = df["step_4"] > 0
df

In [ ]:
# (๗) ในกรณีที่คำนวณตาม (๖) แล้ว ปรากฏว่าพรรคการเมืองทุกพรรคได้รับจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อรวมกันแล้วเกินหนึ่งร้อยคน ให้ดำเนินการคำนวณปรับจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อใหม่ โดยคำนวณตามอัตราส่วนที่ทุกพรรคจะได้รับการจัดสรร
# จำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ ซึ่งเมื่อรวมแล้วไม่เกินหนึ่งร้อยคน โดยให้นำจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อที่แต่ละพรรคจะได้รับคูณด้วยหนึ่งร้อย หารด้วยผลบวกของ
# หนึ่งร้อยกับจำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อที่เกินจำนวนหนึ่งร้อย และให้นำ (๕) มาใช้
# ในการคำนวณด้วยโดยอนุโลม

df_elig_proportion = df[df["step_6_eligible_for_proportion"]]
df_elig_proportion["step_5_scaled_porportion"] = df_elig_proportion["step_4"] / df_elig_proportion["step_4"].sum() * N_PROPORTION_SEATS
df_elig_proportion

In [ ]:
# (๕) ภายใต้บังคับ (๖) ให้จัดสรรจำนวนสมาชิกสภาผู้แทนราษฎร
# แบบบัญชีรายชื่อที่พรรคการเมืองจะได้รับให้ครบหนึ่งร้อยคน โดยจัดสรรให้พรรคการเมืองตามผลลัพธ์
# ตาม (๔) เป็นจำนวนเต็มก่อน...

df_elig_proportion["step_5_full_seats"] = df_elig_proportion["step_5_scaled_porportion"].apply(floor)
df_elig_proportion["step_5_remainder"] = df_elig_proportion["step_5_scaled_porportion"].apply(lambda n: n % 1)
df_elig_proportion

In [ ]:
# (cont from 5) ...หากยังไม่ครบหนึ่งร้อยคน 
# ให้พรรคการเมืองที่มีเศษจากการคำนวณมากที่สุดได้รับการจัดสรรจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อเพิ่มอีกหนึ่งคนตามลำดับจนครบจำนวนหนึ่งร้อยคน ในกรณีที่
# มีเศษเท่ากันให้ดำเนินการตาม (๗)

assigned_proportion_seats = df_elig_proportion["step_5_full_seats"].sum()
n_senators_to_assign_by_remainder = N_PROPORTION_SEATS - assigned_proportion_seats

n_senators_to_assign_by_remainder

In [ ]:
parties_to_get_reps_from_remainder = df_elig_proportion.sort_values("step_5_remainder", ascending=False).head(n_senators_to_assign_by_remainder).index

parties_to_get_reps_from_remainder

In [ ]:
df_elig_proportion["step_5_remainder_seats"] = 0
df_elig_proportion.loc[df_elig_proportion.index.isin(parties_to_get_reps_from_remainder), "step_5_remainder_seats"] = 1
df_elig_proportion

In [ ]:
df_elig_proportion["step_5_proportional_seats"] = df_elig_proportion["step_5_full_seats"] + df_elig_proportion["step_5_remainder_seats"]

df_elig_proportion

In [ ]:
df = df.join(df_elig_proportion.loc[:, ["step_5_proportional_seats"]]).fillna(0)
df["total_seats"] = df["n_constituency_seats"] + df["step_5_proportional_seats"]

In [22]:
df[["n_votes", "total_seats"]]

,n_votes,total_seats
party,,
เพื่อไทย,7881006,136.0
พลังประชารัฐ,8441274,117.0
อนาคตใหม่,6330617,82.0
ประชาธิปัตย์,3959358,53.0
ภูมิใจไทย,3734459,51.0
เสรีรวมไทย,824284,10.0
ชาติไทยพัฒนา,783689,10.0
ประชาชาติ,481490,7.0
เศรษฐกิจใหม่,486273,6.0
